- Scikit pipeline?
- Napravi grader da proceni koliko je tezak primer i primeni na svim
- Import podataka -> Train/Valid split N puta -> Generisanje suma samo za train (da bi mogli da se uverimo da zaista daje dobre rezultate, ako poremetimo valid set mozda greskom dobijemo dobre ili lose rez.) -> 
    Transformacije (SkewTest+Norm/Stand, SMOTE, Method specific transf) - train na train, apply na valid ->
    Trening na train, validacija na valid -> viz i cuvanje
- Sacuvaj - za svaku kombinaciju transf, modela i generisanje suma po jedan entry u pd da bi uzeli najbolje
- Vizuelizacije - Preciznost standardna za svaki difficulty + broj pogodjenih?, za difficulty-e, grafikon za top 5-10 metoda kao linija, grafikon gde su outlieri x-ovi ali zeleni su dobro a crveni lose klasifikovani
- Ako izbacujes ispisi koliko lakih, srednjih, teskih je izbaceno

# Import

In [ ]:
%%capture
%pip install kagglehub openpyxl imbalanced-learn seaborn torch

In [ ]:
import kagglehub as kg
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import VotingClassifier

# Load data

In [ ]:
path = kg.dataset_download("muratkokludataset/dry-bean-dataset")
file_name = '/Dry_Bean_Dataset/Dry_Bean_Dataset.xlsx'
print("Downloaded at: ", path)
data = pd.read_excel(path+file_name)
data